In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )

import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.ndimage.interpolation import shift
from scipy.signal import decimate
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import sem
import pandas as pd
import pickle
from matplotlib import colors, cm
from linear_regression.get_time_stamps_for_regression import remove_manipulation_days



In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [4]:
experiment_record = pd.read_csv('W:\\photometry_2AC\\experimental_record.csv')
experiment_record['date'] = experiment_record['date'].astype(str)
clean_experiments = remove_manipulation_days(experiment_record)

In [5]:
param_names = ['ipsi choices', 'contra choices', 'cues', 'rewards']

In [6]:
ipsi_choice_kernel = []


In [7]:
mouse_ids = ['SNL_photo29', 'SNL_photo30', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']

In [8]:
experiments_to_process = clean_experiments[(clean_experiments['mouse_id'].isin(mouse_ids) ) & (clean_experiments['recording_site'] == 'Nacc')].reset_index(drop=True)

In [9]:
i = []
for mouse in np.unique(mouse_ids):
    i.append(experiments_to_process[experiments_to_process['mouse_id'] == mouse][0:3].index)

In [10]:
flattened_i = [val for sublist in i for val in sublist]

In [11]:
np.unique(experiments_to_process['mouse_id'])

array(['SNL_photo29', 'SNL_photo30', 'SNL_photo32', 'SNL_photo33',
       'SNL_photo34', 'SNL_photo35'], dtype=object)

In [19]:
experiments_to_process = experiments_to_process.loc[flattened_i].reset_index(drop=True)

In [20]:
for index, experiment in experiments_to_process.iterrows():
    mouse = experiment['mouse_id']
    date = experiment['date']
    saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
    save_filename = mouse + '_' + date + '_' + 'linear_regression_kernels_no_repeated_cues_both_cues.p'
    with open(saving_folder + save_filename, "rb") as f:
        session_kernels = pickle.load(f)
        if index == 0:
            ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
            contra_choice_kernel = session_kernels['kernels']['contra choices']
            cue_kernel = session_kernels['kernels']['cues']
            reward_kernel = session_kernels['kernels']['rewards']
            time_stamps = session_kernels['time_stamps']
        else:
            ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
            contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
            cue_kernel = np.vstack([cue_kernel, session_kernels['kernels']['cues']])
            reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])



KeyError: 'cues'

In [18]:
session_kernels['kernels']['ipsi choices']

array([-0.39122325, -0.40139884, -0.40927499, -0.41716961, -0.42460989,
       -0.42972417, -0.43400239, -0.43721555, -0.43982086, -0.44245568,
       -0.44428064, -0.44508897, -0.44558548, -0.44608807, -0.44620482,
       -0.44687155, -0.44761923, -0.44897471, -0.45094252, -0.45357219,
       -0.45681564, -0.4604175 , -0.46439208, -0.46871741, -0.47314171,
       -0.47762936, -0.48212721, -0.48642651, -0.49048436, -0.49426687,
       -0.49776839, -0.50087657, -0.50354518, -0.50573398, -0.50739372,
       -0.50855962, -0.50889294, -0.50515901, -0.5040523 , -0.50032343,
       -0.4956131 , -0.48467588, -0.47922374, -0.46955181, -0.46168392,
       -0.44892046, -0.43841461, -0.4258224 , -0.41207287, -0.39571637,
       -0.3808517 , -0.36737899, -0.35171481, -0.33388949, -0.31857025,
       -0.30158917, -0.28079457, -0.25806802, -0.24009107, -0.22160321,
       -0.2010721 , -0.18383201, -0.15996998, -0.14251037, -0.12496726,
       -0.10618163, -0.08667529, -0.0693399 , -0.05395485, -0.03

In [ ]:
mean_kernels = {}
mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
mean_kernels['cues'] = np.mean(cue_kernel, axis=0)
mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)

In [ ]:
sem_kernels = {}
sem_kernels['ipsi choices'] = sem(ipsi_choice_kernel, axis=0)
sem_kernels['contra choices'] = sem(contra_choice_kernel, axis=0)
sem_kernels['cues'] = sem(cue_kernel, axis=0)
sem_kernels['rewards'] = sem(reward_kernel, axis=0)

In [44]:
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
for param_num, param_name in enumerate(param_names):
    param_kernel = mean_kernels[param_name]
    axs[param_num].plot(time_stamps, param_kernel, label=param_name)
    axs[param_num].axvline(0, color='k')
    axs[param_num].fill_between(time_stamps, param_kernel - sem_kernels[param_name] ,  param_kernel + sem_kernels[param_name], alpha=0.5,
                            facecolor='#7FB5B5', linewidth=0)
    axs[param_num].set_title(param_name)
    axs[param_num].set_xlabel('Time (s)', fontsize=12)
plt.tight_layout()

In [15]:
plot_filename = 'SNL_photo16_17_18_21_22_26_mean_regression_kernels.pdf' 
plot_save_folder = 'W:\\photometry_2AC\\plots\\Linear regression\\'
plt.savefig(plot_save_folder + plot_filename, transparent=True, optimize=True)

In [100]:
experiments_record_tail = experiment_record[experiment_record['recording_site'] == 'tail']

,mouse_id,date,recording_site,fiber_side
28,SNL_photo18,20200223,tail,left
30,SNL_photo18,20200226,tail,left
32,SNL_photo18,20200228,tail,left
33,SNL_photo18,20200229,tail,left
35,SNL_photo18,20200303,tail,left
37,SNL_photo18,20200305,tail,left
39,SNL_photo18,20200307,tail,left
40,SNL_photo18,20200310,tail,left
42,SNL_photo18,20200312,tail,left
43,SNL_photo18,20200313,tail,left


In [99]:
for mouse_id in experiment_record_tail['mouse_id'].unique():
    experiments = experiment_record_tail[experiment_record_tail['mouse_id'] == mouse_id]
    for index, experiment in experiments.iterrows():
        mouse = experiment['mouse_id']
        date = experiment['date']
        saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
        save_filename = mouse + '_' + date + '_' + 'linear_regresion_kernels.p'
        with open(saving_folder + save_filename, "rb") as f:
            session_kernels = pickle.load(f)
            if index == 1:
                ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
                contra_choice_kernel = session_kernels['kernels']['contra choices']
                cue_kernel = session_kernels['kernels']['cues']
                reward_kernel = session_kernels['kernels']['rewards']
                time_stamps = session_kernels['time_stamps']
            else:
                ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
                contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
                cue_kernel = np.vstack([cue_kernel, session_kernels['kernels']['cues']])
                reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])

    all_kernels={}
    all_kernels['ipsi choices'] = ipsi_choice_kernel
    all_kernels['contra choices'] = contra_choice_kernel
    all_kernels['cues'] = cue_kernel
    all_kernels['rewards'] = reward_kernel
    
    mean_kernels = {}
    mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
    mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
    mean_kernels['cues'] = np.mean(cue_kernel, axis=0)
    mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)
    
    std_kernels = {}
    std_kernels['ipsi choices'] = np.std(ipsi_choice_kernel, axis=0)
    std_kernels['contra choices'] = np.std(contra_choice_kernel, axis=0)
    std_kernels['cues'] = np.std(cue_kernel, axis=0)
    std_kernels['rewards'] = np.std(reward_kernel, axis=0)

    fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(15,8))
    axs[0].set_ylabel('Regression coefficient')
    for param_num, param_name in enumerate(param_names):
        param_kernel = all_kernels[param_name]
        axs[param_num].set_prop_cycle(plt.cycler('color',colours))
        axs[param_num].plot(time_stamps,param_kernel.T)

        axs[param_num].set_title(param_name)
        axs[param_num].set_xlabel('Time (s)')
        axs[param_num].axvline(0.2)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
30
32
33
35
37
39
40
42
43
44


In [94]:
first_bin = np.where(time_stamps == 0.2)
np.squeeze(contra_choice_kernel[:,first_bin])

array([0.87801853, 0.61416135, 1.10047466, 0.68212124, 1.1398374 ,
       0.82871293, 1.29255329, 0.77321524, 0.87099428, 0.92496414,
       0.65088095, 1.45326512, 1.19129577, 1.05429092, 0.95228183,
       0.40851538, 0.80581603, 0.6517777 , 0.15186452, 0.7016423 ,
       0.54395115, 0.51660555, 0.18614848, 0.37883117, 0.04421242,
       0.21309765, 0.49045478, 1.78318808, 0.98105266, 0.748314  ,
       0.5835751 , 0.86904679, 0.8399813 , 0.63625214, 0.7217782 ,
       0.66097678, 0.60119746, 0.58549881])

In [95]:
plt.plot(np.squeeze(contra_choice_kernel[:,first_bin]))

In [96]:
experiments

,mouse_id,date,recording_site,fiber_side
28,SNL_photo18,20200223,tail,left
30,SNL_photo18,20200226,tail,left
32,SNL_photo18,20200228,tail,left
33,SNL_photo18,20200229,tail,left
35,SNL_photo18,20200303,tail,left
37,SNL_photo18,20200305,tail,left
39,SNL_photo18,20200307,tail,left
40,SNL_photo18,20200310,tail,left
42,SNL_photo18,20200312,tail,left
43,SNL_photo18,20200313,tail,left


In [97]:
contra_choice_kernel.shape

(38, 301)